# Pandas Tips & Tricks

This notebook presents various tricks to manipulate your data, which are typically non-obvious to a novice in Pandas and data science.

## Reading Data

## Writing Results

## Filtering Rows

## Manipulating Columns
### Adding or Replacing Columns
### Renaming Columns

## Counting

To visualze data in bar or other magnitude charts, you have to count subsets of your raw data.

## String Manipulation